# Introduction to SQLite &
Selecting Sources from the Sloan Digital Sky Survey
========

#### Version 0.2

***
By AA Miller 2019 Mar 25  
Updated by G Hosseinzadeh 2025 Apr 9


This notebook provides a quick introduction to [`SQLite`](https://sqlite.org/index.html) a lightweight implementation of a Structured Query Language (SQL) database. One of the incredibly nice things about `SQLite` is the low overhead needed to set up a database (as you will see in a minute). We will take advantage of this low overhead to build a database later in the week.

At the most basic level - databases *store your bytes*, and later *return those bytes* (or a subset of them) when queried. 

They provide a highly efficient means for filtering your bytes (there are many different strategies that the user can employ). 

The backend for most databases is the **Structured Query Language** or SQL, which is a standard declarative language. 

There are many different libraries that implement SQL: MySQL, PostgreSQL, Greenplum, Microsoft SQL server, IBM DB2, Oracle Database, etc. 

The most basic implementation is [`SQLite`](https://www.sqlite.org) a self-contained, SQL database engine. We will discuss `SQLite` further later in the week, but in brief - it is a nice stand alone package that works really well for small problems (such as the example that we are about to encounter). Jupyter has a convenient interface to SQL that we will use here. Just load the SQL extension and connect to a new database. In the future, use the `%sql` (single line) or `%%sql` (entire cell) magic command to run SQL commands in your notebook.

In [ ]:
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [ ]:
%sql sqlite:///intro.db

And just like that - we have now created a new database `intro.db`, with which we can "store bytes" or later "retrieve bytes" once we have added some data to the database. 

*Aside* - note that unlike many SQL libraries, `SQLite` does not require a server and creates an actual database file on your hard drive. This improves portability, but also creates some downsides as well.

## Problem 1) Basic SQL Operations with SQLite

Now we need to create a table and insert some data. Recall that creating a table requires a specification of the table name, the columns in the table, and the data type for each column. Here's an example where I create a table to store info on my pets:


    %%sql
    create table PetInfo(  
                         Name text, 
                         Species text,
                         Age tinyint,
                         FavoriteFood text
                         )

**Problem 1a**

Create a new table in the database called `Students` with columns `Name`, `College`, and `Year`, where `Year` is the year in school.

In [ ]:
%%sql
# complete

Once a table is created, we can use the database to store bytes. If I were to populate my `PetInfo` table I would do the following:

    %%sql
    insert into PetInfo(Name, Species, Age, FavoriteFood) 
                        values ("Rocky", "Dog", 12, "Bo-Nana");
    insert into PetInfo(Name, Species, Age, FavoriteFood) 
                        values ("100 Emoji-Flames Emoji", "Red Panda", 2, "bamboo leaves");

*Note* - column names do not need to be explicitly specified, but for clarity this is always preferred.

**Problem 1b**

Insert data for yourself, and the two people sitting next to you into the database.

In [ ]:
%%sql
# complete

Now that we have bytes in the database, we can retrieve those bytes with one (or several) queries. There are 3 basic building blocks to a query:

    SELECT...
    FROM...
    WHERE...
    
Where `SELECT` specifies the information we want to retrieve from the database, `FROM` specifies the tables being queried in the database, and `WHERE` specifies the conditions for the query. 

**Problem 1c**

Select the institutions for all students in the `Students` table who have been in school for more than 2 years.

In [ ]:
%%sql
# complete

## Problem 2) Complex Queries with SDSS

Above we looked at the most basic operations possible with a database (recall - databases are unnecessary, and possibly cumbersome, with small data sets). A typical database consists of many tables, and these tables may be joined together to unlock complex questions for the data. 

As a reminder on (some of) this functionality, we are now going to go through some problems using the SDSS database. The full [SDSS schema](http://skyserver.sdss.org/dr13/en/help/browser/browser.aspx) explains all of the tables, columns, views and functions for querying the database. We will keep things relatively simple in that regard.

In [ ]:
from astroquery.sdss import SDSS

[`astroquery`](http://astroquery.readthedocs.io/en/latest/) enables seemless connections to the SDSS database via the Python shell.

**Problem 2a**

Select 20 random sources from the [`PhotoObjAll`](https://skyserver.sdss.org/dr13/en/help/docs/tabledesc.aspx?name=PhotoObjAll) table and return all columns in the table. 

*Hint* - while this would normally be accomplished by starting the query `select limit 20 ...`, SDSS CasJobs uses Microsoft's SQL Server, which adopts `select top 20 ...` to accomplish an identical result.

In [ ]:
SDSS.query_sql(  # complete

That's more columns than we will likely ever need. Instead, let's focus on `objID`, a unique identifier, `cModelMag_u`, `cModelMag_g`, `cModelMag_r`, `cModelMag_i`, and `cModelMag_z`, the source magnitude in $u', g', r', i', z'$, respectively.

We will now introduce the concept of joining two tables. 

The most common operation is known as an `inner join` (which is often referred to as just `join`). An `inner join` returns records that have matching sources in both tables in the join. 

Less, but nevertheless still powerful, is the `outer join`. An outer join returns *all* records in either table, with `NULL` values for columns in a table in which the record does not exist.

Specialized versions of the `outer join` include the `left join` and `right join`, whereby *all* records in either the left or right table, respectively, are returned along with their counterparts. 

**Problem 2b**

Select `objid` and $u'g'r'i'z'$ from `PhotoObjAll` and the corresponding `class` from [`specObjAll`](https://skyserver.sdss.org/dr13/en/help/docs/tabledesc.aspx?name=SpecObjAll) for 20 random sources. 

There are multiple columns you could use to join the tables, in this case match `objid` to `bestobjid` from `specObjAll` and use an `inner join`.

In [ ]:
SDSS.query_sql(  # complete

**Problem 2c**

Perform an identical query to the one above, but this time use a `left outer join` (or `left join`).

How do your results compare to the previous query?

In [ ]:
SDSS.query_sql(  # complete

**Problem 2d**

This time use a `right outer join` (or `right join`).

How do your results compare to the previous query?

In [ ]:
SDSS.query_sql(  # complete

## Challenge Problem

To close the notebook we will perform a nested query. In brief, the idea is to join the results of one query with a separate query.

Here, we are going to attempt to identify bright AGN that don't have SDSS spectra. To do so we will need the `photoObjAll` table, the `specObjAll` table, and the `rosat` table, which includes all cross matches between SDSS sources and X-ray sources detected by the [Rosat satellite](https://heasarc.gsfc.nasa.gov/docs/rosat/rosat3.html).

Create a nested query that selects all *Rosat* sources that don't have SDSS spectra with `cModelFlux_u + cModelFlux_g + cModelFlux_r + cModelFlux_i + cModelFlux_z > 10000` (this flux contraint ensures the source is bright without making any cuts on color) and `type = 3`, this last constraint means the source is extended in SDSS images. 

*Hint* - you may run into timeout issues in which case you should run the query on CasJobs.

In [ ]:
SDSS.query_sql(  # complete